In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
from glob import glob
import cv2
import numpy as np

In [4]:
# Load labeled frames and their corresponding labels
def load_labeled_frames(data_folder):
    images = []
    labels = []

    for image_path in glob(os.path.join(data_folder, '*.jpg')):
        image = cv2.imread(image_path)
        image = cv2.resize(image, (20, 20))

        label = os.path.splitext(os.path.basename(image_path))[0].split('_')[-1]

        images.append(image)

        if label == '0':
            labels.append(0)
        elif label == '1':
            labels.append(1)

    return images, labels

# Preprocess the data
def preprocess_data(images, labels):
    images = np.array(images)
    labels = np.array(labels)

    # Normalize pixel values to be between 0 and 1
    images = images / 255.0

    # Shuffle and split the data into training and testing sets
    images, labels = shuffle(images, labels, random_state=46)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=45)

    return X_train, X_test, y_train, y_test

In [5]:
# Example usage
output_folder = 'labeled_frames_binary_test'

# Load labeled frames and their corresponding labels
images, labels = load_labeled_frames(output_folder)

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_data(images, labels)

input_shape = X_train[0].shape

In [5]:
y_train

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0])

In [7]:
# Build the model
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid')) 

# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',  # Use binary crossentropy for binary classification
              metrics=['accuracy', 'AUC'])

# Train the model
history = model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test))


Epoch 1/40
1/1 [==============================] - 8s 8s/step - loss: 0.7095 - accuracy: 0.2105 - auc: 0.0667 - val_loss: 0.6420 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 2/40
1/1 [==============================] - 0s 239ms/step - loss: 0.6652 - accuracy: 0.6316 - auc: 0.8444 - val_loss: 0.5909 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 3/40
1/1 [==============================] - 0s 171ms/step - loss: 0.6292 - accuracy: 0.6316 - auc: 0.9111 - val_loss: 0.5487 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 4/40
1/1 [==============================] - 0s 204ms/step - loss: 0.6016 - accuracy: 0.6316 - auc: 0.9111 - val_loss: 0.5155 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 5/40
1/1 [==============================] - 0s 178ms/step - loss: 0.5845 - accuracy: 0.6316 - auc: 0.9111 - val_loss: 0.4912 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 6/40
1/1 [==============================] - 0s 170ms/step - loss: 0.5749 - accuracy: 0.6316 - auc: 0.9111 - val_loss: 0.4729 - val_accura

In [8]:
# Evaluate the model
test_loss, test_acc, test_auc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest AUC:', test_auc)

1/1 - 0s - loss: 0.0172 - accuracy: 1.0000 - auc: 1.0000 - 57ms/epoch - 57ms/step

Test AUC: 1.0


In [17]:
import cv2
import numpy as np

# Open the video file
video_path = './vidscapstone/vid9.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

rois = [(1374, 773, 27, 27), (458, 283, 27, 27), (1403, 280, 23, 24), (504, 824, 23, 22)]

# Loop through frames
while True:
    # Read a frame
    ret, frame = cap.read()

    # Break the loop if the video is finished
    if not ret:
        break

    count = 0

    for roi in rois:
        x, y, w, h = roi
        roi_processed = frame[y:y+h, x:x+w]
        roi_processed = cv2.resize(roi_processed, (20, 20))
        roi_processed = cv2.convertScaleAbs(roi_processed)
        roi_processed = np.array(roi_processed)
        roi_processed = roi_processed / 255.0
        roi_processed = np.expand_dims(roi_processed, axis=0)
        pred = model.predict(roi_processed)
        print(pred)
        if pred > 0.5:
            count += 1

    #frame_to_show = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    #cv2.imshow(f'Turned on lights {count}', frame_to_show)
    
    #show each roi
    for roi in rois:
        x, y, w, h = roi
        cv2.imshow(f'ROI {x}, {y}', frame[y:y+h, x:x+w])

        breaker = False
        # Press q to exit
        if cv2.waitKey(0) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            breaker = True
            break
        else:
            cv2.destroyAllWindows()

    if breaker:
        break

    


# Release the video capture object
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


1/1 [==============================] - 0s 38ms/step
[[0.9999981]]
1/1 [==============================] - 0s 22ms/step
[[0.9999987]]
1/1 [==============================] - 0s 23ms/step
[[0.9997089]]
1/1 [==============================] - 0s 23ms/step
[[0.06282663]]
1/1 [==============================] - 0s 25ms/step
[[0.9971877]]
1/1 [==============================] - 0s 22ms/step
[[0.99999917]]
1/1 [==============================] - 0s 25ms/step
[[0.9997112]]
1/1 [==============================] - 0s 24ms/step
[[0.99967235]]


In [ ]:
import pickle

directory = 'testing_binary_class_classification/'

model.save(directory + 'testing_binary_class_classification_model_Adam.h5')

with open(directory + 'testing_binary_class_classification_model_Adam_history', 'wb') as file_pi:
    pickle.dump(history, file_pi)

with open(directory + 'testing_binary_class_classification_model_Adam_X_test', 'wb') as file_pi:
    pickle.dump(X_test, file_pi)

with open(directory + 'testing_binary_class_classification_model_Adam_y_test', 'wb') as file_pi:
    pickle.dump(y_test, file_pi)